## 1.Connect to Elasticsearch

In [1]:
from elasticsearch_dsl import connections
client = connections.create_connection(hosts=["10.4.177.69"],
                     port=9200,
                     http_auth=('elastic', 'elastic'),
                     ca_certs='http_ca.crt',
                     use_ssl=False,
                     verify_certs=True)

## 2. Import Data and create an Elasticsearch index

In [2]:
import json

with open('companies.json') as f:
    records = json.load(f)
print(len(records))

15


In [3]:
from elasticsearch.helpers import bulk

resp = bulk(client, records, index = "companies")

## 3. Run elasticsearch-dsl queries

### 3.1 Which company is the oldest?

In [4]:
from elasticsearch_dsl import Search

s = Search(using=client, index="companies")
a1 = s.sort('founded')
response1 = a1[0:1].execute()
print(response1[0].company)

Allianz


### 3.2 Which Company's product is a search engine?

In [6]:
s = Search(using=client, index="companies")
a2 = s.query("match", description="search engine")
response2 = a2.execute()
for hit in response2:
    print(hit.company)

Alphabet


### 3.3 Which companies are headquartered in the USA?

In [7]:
s = Search(using=client, index="companies")
a3 = s.query("match", country="USA")
response3 = a3.execute()
for hit in response3:
    print(hit.company)

Apple
Microsoft
Amazon
Alphabet
Tesla


### 3.4 WHich companies have been founded in 1990s?

In [8]:
s = Search(using=client, index="companies")
a4 = s.filter('range', founded={"gte": 1990, "lt": 2000})
response4 = a4.execute()
for hit in response4:
    print(hit.company)

Amazon
Alphabet
Alibaba Group


### 3.5 What is the total annual revenue of all companies in Asia?

In [10]:
s = Search(using=client, index="companies")
a5 = s.query('match', country='China South Korea Japan')
a5.aggs.metric('total_revenue', 'sum', field="revenue")
response5 = a5.execute()
total_annual = response5.aggregations.total_revenue.value
print(total_annual)

964.3399963378906
